In [1]:
from pathlib import Path
from mesh_handler import meshes_to_xdmf
from preprocessing_utils import convert_xdmf_to_torch, torch_to_meshes, compute_edge_weights
import torch
from tqdm import tqdm

In [2]:
CWD = Path.cwd()
test_data_dir = CWD / "4Students_Validation_cases_cropped"
cleaned_test_dir = CWD / "data_cleaned_test"

convert_xdmf_to_torch(test_data_dir, cleaned_test_dir)

  0%|          | 0/2 [00:00<?, ?it/s]

167.xdmf


 50%|█████     | 1/2 [00:01<00:01,  1.77s/it]

160.xdmf


100%|██████████| 2/2 [00:02<00:00,  1.20s/it]


In [3]:
mesh_fps = list(cleaned_test_dir.glob("*.pth"))
test_ids = [fp.name.split("_")[1].split(".")[0] for fp in mesh_fps]
test_ids

['160', '167']

In [4]:
REPLACE_INFLOW = True
EDGES_WEIGHTS = True
Path.mkdir(CWD / "results", exist_ok=True, parents=True)
results_dir = CWD / "results"

def mock_model(X_nodes, X_edges, edges_weights=None):
    return torch.ones((X_nodes.shape[0], 4))
model = mock_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)

for i, mesh_id in enumerate(test_ids):
    print("evaluating mesh",i, mesh_id)
    data = torch.load(cleaned_test_dir / f"mesh_{mesh_id}.pth")
    nodes_feat = data["nodes"]
    edges_index = data["edges"]
    vP_tensor = torch.zeros((nodes_feat.shape[0], nodes_feat.shape[1], 4)).to(device)
    vP_tensor[0,:,:4] = nodes_feat[0,:,3:7]
    vP_tensor[1:,:,:4] = nodes_feat[1,:,3:7]
    vP_tensor[2:,:,:4] = nodes_feat[2:,:,3:7]

    X_init = nodes_feat[0,:,:]
    N_nodes = X_init.shape[0]
    wall_mask = torch.norm(X_init, p=2, dim=1, keepdim=True) > 1e-10  #wall on column 11
    inflow_mask = ((X_init[:, 1] < 1e-2) & (X_init[:,4] > 0)).unsqueeze(-1)  #inflow on column 12
    edges_weights = compute_edge_weights(edges_index, X_init)
    
    for t in tqdm(range(2, nodes_feat.shape[0]-1)):
        if t==2:
            X_nodes_now = nodes_feat[t,:,:]  #pos, speeds, P on col 0 to 6
            X_nodes_past = nodes_feat[t-1,:,:]

        X_nodes_future = nodes_feat[t+1,:,:]
        accelerations = torch.zeros((N_nodes, 3))  #acceleration column 7 to 9
        accelerations[:,:] =  (X_nodes_now[:,3:6] -  X_nodes_past[:,3:6])/0.01
        time_steps=torch.full((N_nodes, 1), t)   #time on column 10
        if REPLACE_INFLOW:
            X_nodes_replace = X_nodes_now.clone()
            X_nodes_replace[inflow_mask.squeeze(-1)] = X_nodes_future[inflow_mask.squeeze(-1)]
            X_nodes_t = torch.cat([X_nodes_replace, accelerations, time_steps, wall_mask, inflow_mask], dim=-1)
        else:
            X_nodes_t = torch.cat([X_nodes_now, accelerations, time_steps, wall_mask, inflow_mask], dim=-1)
        if EDGES_WEIGHTS:
            out = model(X_nodes_t.to(device), edges_index.to(device), edges_weights.to(device))  #(n_nodes, 4)  4 being (vx, vy, vz, P)
        else:
            out = model(X_nodes_t.to(device), edges_index.to(device))  #(n_nodes, 4)  4 being (vx, vy, vz, P)
        vP_tensor[t+1,:,:] = out.clone()    #store results
        X_nodes_past = X_nodes_now
        X_nodes_future[:,3:7] = out.clone()  #replace speeds with predicted speeds 
        X_nodes_now = X_nodes_future
    #####TOADAPT###
    src_filename = f"TEST_AllFields_Resultats_MESH_{mesh_id}.xdmf"
    xdmf_file_path = test_data_dir / src_filename
    #################  
    data = torch.load(cleaned_test_dir / f"mesh_{mesh_id}.pth")
    meshes = torch_to_meshes(vP_tensor, xdmf_file_path)
    out_xdmf_file_path = results_dir / src_filename
    meshes_to_xdmf(out_xdmf_file_path, meshes)

evaluating mesh 0 160


100%|██████████| 77/77 [00:00<00:00, 825.38it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jingzhan/DC_blood/demo/4Students_Validation_cases_cropped/TEST_AllFields_Resultats_MESH_160.xdmf'